### Step 2. Mapping functional genomics data to personalized genomes 

Add detailed explanation here

In [ ]:
# provide the sample name and the fastq file to be mapped here 
sample="MM031"
atac_fastq="/staging/leuven/stg_00002/lcb/zkalender/melanoma_ATAC/00.FASTQ/MM031_BL/OmniATAC__MM031__untreated_S3_R2_001.fastq.gz"

In [ ]:
# create necessary folders 
mkdir -p /staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC
mkdir -p /staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP1_ind_map
mkdir -p /staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP2_ind_map

cd /staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC

In [ ]:
# Map to the reference genome
# map
bowtie2 -p 20 --very-sensitive \
-x /staging/leuven/stg_00002/lcb/zkalender/resources/refdata-GRCh38-2.1.0/fasta/genome \
-U ${atac_fastq}  \
 | samtools view -b - | samtools sort -@ 20 -l 1 -O bam \
-o /staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/REF/${sample}.ATAC.REF.sorted.bam

samtools flagstat \
/staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/REF/${sample}.ATAC.REF.sorted.bam \
> /staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/REF/${sample}.ATAC.REF.sorted.bam.flagstat

# duplicate removal
picard MarkDuplicates \
I=/staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/REF/${sample}.ATAC.REF.sorted.bam \
O=/staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/REF/${sample}.ATAC.REF.sorted.dedup.bam \
CREATE_INDEX=true \
VALIDATION_STRINGENCY=SILENT \
REMOVE_DUPLICATES=TRUE \
M=/staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/REF/${sample}.ATAC.REF.dedup.metrics

samtools view -@ 10 -q 2 -b /staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/REF/${sample}.ATAC.REF.sorted.dedup.bam  |\
 samtools sort - -@ 10 -O bam -l 9 -T ${VSC_SCRATCH}/tmp -o /staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/REF/${sample}.ATAC.REF.sorted.dedup.Q2.bam

samtools flagstat /staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/REF/${sample}.ATAC.REF.sorted.dedup.Q2.bam > \
/staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/REF/${sample}.ATAC.REF.sorted.dedup.Q2.flagstat

# peak calling
mkdir -p /staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/REF/q005_peaks

macs2 callpeak \
-t /staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/REF/${sample}.ATAC.REF.sorted.dedup.Q2.bam \
 -q 0.05 \
-n /staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/REF/q005_peaks/${sample}.ATAC.REF.q_005.peaks \
--nomodel --shift -75 --extsize 150 --keep-dup all --call-summits \
-f BAM

intersectBed -a /staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/REF/q005_peaks/${sample}.ATAC.REF.q_005.peaks_summits.bed \
-b /staging/leuven/stg_00002/lcb/zkalender/resources/hg38/hg38.canonical_chromosomes.bed \
-wa -nonamecheck | sort -u | sortBed -i stdin > \
/staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/REF/q005_peaks/${sample}.ATAC.REF.q_005.peaks_summits_filt.bed



In [ ]:
# map to HAP1 
bowtie2 -p 20 --very-sensitive \
-x /staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/liftover_proc/bowtie2_index/${sample}.hap1.fa \
-U ${atac_fastq}  \
 | samtools view -b - | samtools sort -@ 20 -l 1 -O bam \
-o /staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP1_ind_map/${sample}.ATAC.HAP1_ind_map.sorted.bam

samtools flagstat /staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP1_ind_map/${sample}.ATAC.HAP1_ind_map.sorted.bam > \
/staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP1_ind_map/${sample}.ATAC.HAP1_ind_map.sorted.bam.flagstat

# duplicate removal
picard MarkDuplicates \
I=/staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP1_ind_map/${sample}.ATAC.HAP1_ind_map.sorted.bam \
O=/staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP1_ind_map/${sample}.ATAC.HAP1_ind_map.sorted.dedup.bam \
CREATE_INDEX=true \
VALIDATION_STRINGENCY=SILENT \
REMOVE_DUPLICATES=TRUE \
M=/staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP1_ind_map/${sample}.ATAC.HAP1_ind_map.dedup.metrics

# quality filter
samtools view -@ 10 -q 2 -b /staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP1_ind_map/${sample}.ATAC.HAP1_ind_map.sorted.dedup.bam  |\
 samtools sort - -@ 10 -O bam -l 9 -T ${VSC_SCRATCH}/tmp -o /staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP1_ind_map/${sample}.ATAC.HAP1_ind_map.sorted.dedup.Q2.bam

samtools flagstat /staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP1_ind_map/${sample}.ATAC.HAP1_ind_map.sorted.dedup.Q2.bam > \
/staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP1_ind_map/${sample}.ATAC.HAP1_ind_map.sorted.dedup.Q2.flagstat

# call peaks on reads mapped to HAP1
mkdir -p /staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP1_ind_map/q005_peaks
macs2 callpeak \
-t /staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP1_ind_map/${sample}.ATAC.HAP1_ind_map.sorted.dedup.Q2.bam \
 -q 0.05 \
-n /staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP1_ind_map/q005_peaks/${sample}.ATAC.HAP1_ind_map.q_005.peaks \
--nomodel --shift -75 --extsize 150 --keep-dup all --call-summits \
-f BAM

In [ ]:
# map to HAP2 
bowtie2 -p 20 --very-sensitive \
-x /staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/liftover_proc/bowtie2_index/${sample}.hap2.fa \
-U ${atac_fastq}  \
 | samtools view -b - | samtools sort -@ 20 -l 1 -O bam \
-o /staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP2_ind_map/${sample}.ATAC.HAP2_ind_map.sorted.bam

samtools flagstat /staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP2_ind_map/${sample}.ATAC.HAP2_ind_map.sorted.bam > \
/staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP2_ind_map/${sample}.ATAC.HAP2_ind_map.sorted.bam.flagstat

# duplicate removal
picard MarkDuplicates \
I=/staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP2_ind_map/${sample}.ATAC.HAP2_ind_map.sorted.bam \
O=/staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP2_ind_map/${sample}.ATAC.HAP2_ind_map.sorted.dedup.bam \
CREATE_INDEX=true \
VALIDATION_STRINGENCY=SILENT \
REMOVE_DUPLICATES=TRUE \
M=/staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP2_ind_map/${sample}.ATAC.HAP2_ind_map.dedup.metrics

# quality filter
samtools view -@ 10 -q 2 -b /staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP2_ind_map/${sample}.ATAC.HAP2_ind_map.sorted.dedup.bam  |\
 samtools sort - -@ 10 -O bam -l 9 -T ${VSC_SCRATCH}/tmp \
 -o /staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP2_ind_map/${sample}.ATAC.HAP2_ind_map.sorted.dedup.Q2.bam

samtools flagstat /staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP2_ind_map/${sample}.ATAC.HAP2_ind_map.sorted.dedup.Q2.bam > \
/staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP2_ind_map/${sample}.ATAC.HAP2_ind_map.sorted.dedup.Q2.flagstat

# call peaks on reads mapped to HAP2
mkdir -p /staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP2_ind_map/q005_peaks
macs2 callpeak \
-t /staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP2_ind_map/${sample}.ATAC.HAP2_ind_map.sorted.dedup.Q2.bam \
 -q 0.05 \
-n /staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP2_ind_map/q005_peaks/${sample}.ATAC.HAP2_ind_map.q_005.peaks \
--nomodel --shift -75 --extsize 150 --keep-dup all --call-summits \
-f BAM

In [ ]:
## Consolidate peaks

liftOver /staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP1_ind_map/q005_peaks/${sample}.ATAC.HAP1_ind_map.q_005.peaks_summits.bed \
/staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/liftover_proc/${sample}.hap1_to_hg38.over.chain \
/staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP1_ind_map/q005_peaks/${sample}.ATAC.HAP1_ind_map.q_005.peaks_summits.hg38.bed \
/staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP1_ind_map/q005_peaks/${sample}.ATAC.HAP1_ind_map.unmapp

liftOver /staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP2_ind_map/q005_peaks/${sample}.ATAC.HAP2_ind_map.q_005.peaks_summits.bed \
/staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/liftover_proc/${sample}.hap2_to_hg38.over.chain \
/staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP2_ind_map/q005_peaks/${sample}.ATAC.HAP2_ind_map.q_005.peaks_summits.hg38.bed \
/staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP2_ind_map/q005_peaks/${sample}.ATAC.HAP2_ind_map.unmapp


# extend to 500 bp & filter for blacklisted regions
# HAP1
slopBed -i \
/staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP1_ind_map/q005_peaks/${sample}.ATAC.HAP1_ind_map.q_005.peaks_summits.hg38.bed \
-b 250 -g /staging/leuven/stg_00002/lcb/resources/human/hg38/hg38.canonical.chrom.sizes | \
subtractBed -a stdin -b /staging/leuven/stg_00002/lcb/zkalender/resources/hg38/peakPass60Perc_sorted.bed -A -wa \
> /staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP1_ind_map/q005_peaks/${sample}.ATAC.HAP1_ind_map.q_005.peaks_summits.hg38.extended.bed

# HAP2
slopBed -i  /staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP2_ind_map/q005_peaks/${sample}.ATAC.HAP2_ind_map.q_005.peaks_summits.hg38.bed \
-b 250 -g /staging/leuven/stg_00002/lcb/resources/human/hg38/hg38.canonical.chrom.sizes | \
subtractBed -a stdin -b /staging/leuven/stg_00002/lcb/zkalender/resources/hg38/peakPass60Perc_sorted.bed -A -wa > \
/staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP2_ind_map/q005_peaks/${sample}.ATAC.HAP2_ind_map.q_005.peaks_summits.hg38.extended.bed

# REF
slopBed -i  /staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/REF/q005_peaks/${sample}.ATAC.REF.q_005.peaks_summits_filt.bed \
-b 250 -g /staging/leuven/stg_00002/lcb/resources/human/hg38/hg38.canonical.chrom.sizes | \
subtractBed -a stdin -b /staging/leuven/stg_00002/lcb/zkalender/resources/hg38/peakPass60Perc_sorted.bed -A -wa > \
/staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/REF/q005_peaks/${sample}.ATAC.REF.q_005.peaks_summits.extended.bed

cat /staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP1_ind_map/q005_peaks/${sample}.ATAC.HAP1_ind_map.q_005.peaks_summits.hg38.extended.bed | \
tr '/' '\t' | cut -f1-3,15,16 > \
/staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP1_ind_map/q005_peaks/${sample}.ATAC.HAP1_ind_map.q_005.peaks_summits.hg38.extended.renamed.bed

cat /staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP2_ind_map/q005_peaks/${sample}.ATAC.HAP2_ind_map.q_005.peaks_summits.hg38.extended.bed | \
tr '/' '\t' | cut -f1-3,15,16 > \
/staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP2_ind_map/q005_peaks/${sample}.ATAC.HAP2_ind_map.q_005.peaks_summits.hg38.extended.renamed.bed

cat /staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/REF/q005_peaks/${sample}.ATAC.REF.q_005.peaks_summits.extended.bed | \
tr '/' '\t' | cut -f1-3,15,16 > \
/staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/REF/q005_peaks/${sample}.ATAC.REF.q_005.peaks_summits.extended.renamed.bed

# normalize peak scores
module load R/3.4.1-foss-2014a-noX
Rscript ~/lcb/zkalender/software/src_zkalender/CDK7_Menin/normalize_fixed_width_peaks.R \
/staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP1_ind_map/q005_peaks/${sample}.ATAC.HAP1_ind_map.q_005.peaks_summits.hg38.extended.renamed.bed \
/staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP1_ind_map/q005_peaks/${sample}.ATAC.HAP1_ind_map.q_005.peaks_summits.hg38.extended_normalized.bed

Rscript ~/lcb/zkalender/software/src_zkalender/CDK7_Menin/normalize_fixed_width_peaks.R \
/staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP2_ind_map/q005_peaks/${sample}.ATAC.HAP2_ind_map.q_005.peaks_summits.hg38.extended.renamed.bed \
/staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP2_ind_map/q005_peaks/${sample}.ATAC.HAP2_ind_map.q_005.peaks_summits.hg38.extended_normalized.bed

Rscript ~/lcb/zkalender/software/src_zkalender/CDK7_Menin/normalize_fixed_width_peaks.R \
/staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/REF/q005_peaks/${sample}.ATAC.REF.q_005.peaks_summits.extended.renamed.bed \
/staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/REF/q005_peaks/${sample}.ATAC.REF.q_005.peaks_summits.extended_normalized.bed


module load gawk/4.1.3
module load bedtools/20181008-foss-2014a

cat /staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP1_ind_map/q005_peaks/${sample}.ATAC.HAP1_ind_map.q_005.peaks_summits.hg38.extended_normalized.bed \
/staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/HAP2_ind_map/q005_peaks/${sample}.ATAC.HAP2_ind_map.q_005.peaks_summits.hg38.extended_normalized.bed \
/staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/REF/q005_peaks/${sample}.ATAC.REF.q_005.peaks_summits.extended_normalized.bed | \
/staging/leuven/stg_00002/lcb/ghuls/software/iterative_peak_filtering/iterative_peak_filtering.sh - /staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/${sample}/AS_ATAC/${sample}_REF_HAP1_HAP2_peaks.merged_and_filtered.bed \
/staging/leuven/stg_00002/lcb/resources/human/hg38/hg38.canonical.chrom.sizes